In [ ]:
import pandas as pd

In [23]:
initial_dataset = pd.read_csv("biodegradable_a.csv")

# Total with means
total_categorical_dataset = initial_dataset.drop() # TODO 
total_categorical_dataset = total_categorical_dataset.fillna(total_categorical_dataset.mode())

total_numerical_dataset = initial_dataset.drop() # TODO 
total_numerical_dataset = total_numerical_dataset.fillna(total_numerical_dataset.mean())
total_dataset = total_categorical_dataset + total_numerical_dataset

total_len, _ = total_dataset.shape
model_dataset_len = round(total_len * 0.25)

dataset = total_dataset[0:model_dataset_len]
dataset_ivs = total_dataset[model_dataset_len:total_len]

print(total_dataset.shape)

# Removal of None/NaN vals
dataset_dropna = initial_dataset.dropna()

dropna_len, _ = dataset_dropna.shape
model_dropna_len = round(total_len * 0.25)

dropna = dataset_dropna[0:model_dropna_len]
dropna_ivs = dataset_dropna[model_dropna_len:dropna_len]

print(dataset_dropna.shape)

(4564, 42)
(889, 42)


C:\Users\rapos\AppData\Local\Temp\ipykernel_26972\3084408616.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  total_dataset = initial_dataset.fillna(initial_dataset.mean())


In [15]:


# https://scikit-learn.org/stable/modules/preprocessing.html

Y_bc = dataset["Biodegradable"]
X_bc = dataset.drop(["Biodegradable", "nHM", "F01", "F04", "NssssC",
"nCb", "nCp", "nO", "F03", "F03_CO", "nN_N", "nArNO2", "nCRX3",
"nCIR", "B01", "B03", "N_073", "B04", "nCrt", "C_026",
"F02_CN", "nHDon", "nN", "nArCOOR", "nX"], axis=1)


#X_train = 
print(X_bc)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#scaler.fit(X_train)
#X_train = scaler.transform(X_train)
#X_test = scaler.transform(X_test)

X_bc.shape

       SpMax_L   J_Dz(e)          C     SdssC    HyWi_B       LOC     SM6_L  \
0     3.919000  2.690900  31.400000  0.000000  3.106000  2.550000  9.002000   
1     4.170000  2.114400  30.800000  0.000000  2.461000  1.393000  8.723000   
2     3.932000  3.251200  26.700000  0.000000  3.279000  2.585000  9.110000   
3     3.000000  2.709800  20.000000  0.000000       NaN  0.918000  6.594000   
4     4.236000  3.394400  29.400000 -0.271000  3.449000  2.753000  9.528000   
...        ...       ...        ...       ...       ...       ...       ...   
4559  4.175650  3.454649  27.300000 -0.308308  2.568631  1.033666  8.617289   
4560  3.853731  2.747142  33.300000  0.000000  2.554222  0.983132  8.099587   
4561  4.294771  3.471226  28.422154 -0.249113  3.026879  1.913110  9.225643   
4562  4.560376  3.896390        NaN -0.228584  3.115647  1.804976  9.521317   
4563  4.045097  2.847185  23.386643  0.000000  2.543924  1.476854  8.499379   

            Me        Mi  SpPosA_B   SpMax_A  Psi_i

(4564, 17)